In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import pandas as pd
from zwad.ad.preprocess import *
from zwad.ad.postprocess import *

In [20]:
datadir = os.path.join('..', 'data')

oid, feature = load_dataset(os.path.join(datadir, 'oid_m31.dat'), os.path.join(datadir, 'feature_m31.dat'))
# Optionally with feature names:
# oid, feature = load_dataset(os.path.join(datadir, 'oid_m31.dat'), os.path.join(datadir, 'feature_m31.dat'), os.path.join(datadir, 'feature_m31.name'))
anomalies_gmm = load_ad_table(os.path.join(datadir, 'm31_gmm.csv'))
anomaly_features = extract_anomaly_features(anomalies_gmm['oid'], oid, feature)

In [21]:
anomaly_features

array([[2.3800001e+00, 2.9370630e-01, 4.8951048e-02, ..., 1.0970883e+00,
        8.7504232e-01, 1.6457632e+01],
       [2.0444999e+00, 3.5802469e-01, 6.1728396e-03, ..., 1.0822589e+00,
        9.0305829e-01, 1.6626255e+01],
       [8.0999947e-01, 3.9370079e-03, 3.9370079e-03, ..., 9.9654287e-02,
        3.1402901e-01, 1.6848158e+01],
       ...,
       [2.0720000e+00, 2.2222222e-01, 4.2735044e-02, ..., 6.2597132e-01,
        7.3348981e-01, 1.6854887e+01],
       [3.2999992e-01, 6.1224490e-02, 8.1632650e-03, ..., 4.4723526e-02,
        5.1094729e-01, 1.6834017e+01],
       [1.3650036e-01, 3.5826772e-01, 3.9370079e-03, ..., 7.0678927e-02,
        8.8457823e-01, 1.5309025e+01]], dtype=float32)